In [ ]:
pip install fredapi

In [ ]:
# Lighthouse Macro Dashboard (3 Panels)
# Dependencies: pandas, numpy, matplotlib, requests
# Sources: FRED (CPI & NAAIM), AlphaVantage (2Y Treasury Yield)

import os, requests, pandas as pd, matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.ticker import FuncFormatter

# API Keys
FRED_API_KEY = "7f8e44038ee69c4f78cf71873e85db16"
ALPHAVANTAGE_API_KEY = "IOTZFZG01XK55BHI"

# Colors
PRIMARY_BLUE = "#0067db"
SUNSET_ORANGE = "#ff7f32"
DEEP_PURPLE = "#5a2d82"
MID_GRAY = "#7f8c8d"
LIGHT_BLUE = "#7bb5ff"

# --- Fetch helpers ---
def fetch_fred(series, start="2000-01-01"):
    url = f"https://api.stlouisfed.org/fred/series/observations?series_id={series}&observation_start={start}&api_key={FRED_API_KEY}&file_type=json"
    r = requests.get(url); r.raise_for_status()
    df = pd.DataFrame(r.json()["observations"])
    df["date"] = pd.to_datetime(df["date"]); df["value"] = pd.to_numeric(df["value"], errors="coerce")
    return df.set_index("date")["value"].dropna()

def fetch_alpha_tsy(maturity="2year"):
    url = f"https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity={maturity}&apikey={ALPHAVANTAGE_API_KEY}"
    r = requests.get(url); r.raise_for_status()
    df = pd.DataFrame(r.json()["data"])
    df["date"] = pd.to_datetime(df["date"]); df["value"] = pd.to_numeric(df["value"], errors="coerce")
    return df.set_index("date")["value"].sort_index()

def yoy(series, months=12):
    return series.pct_change(months) * 100

# --- Data ---
cpi = fetch_fred("CPILFESL")
cpi_yoy = yoy(cpi).dropna()
naaim = fetch_fred("NAAIM") # Changed series ID from "NAAIMEMPEX" to "NAAIM"
ust2y = fetch_alpha_tsy("2year")

# --- Styling ---
def style(ax, title):
    ax.set_facecolor("white"); ax.grid(False)
    for spine in ax.spines.values(): spine.set_visible(False)
    rect = Rectangle((0,0),1,1, transform=ax.transAxes, fill=False, lw=1.2, edgecolor=LIGHT_BLUE)
    ax.add_patch(rect)
    ax.set_title(title, loc="left", fontsize=12, weight="bold")

def annotate_last(ax, s, color, fmt="{:.1f}%"):
    if s.empty: return
    x,y = s.index[-1], s.iloc[-1]
    ax.scatter([x],[y], color=color, s=25)
    ax.annotate(fmt.format(y),(x,y),xytext=(6,0),textcoords="offset points", color=color, fontsize=9, weight="bold")

# --- Plot ---
fig, (ax1, ax2, ax3) = plt.subplots(3,1,figsize=(10,9), dpi=200)
fig.patch.set_facecolor("white")

# Panel 1: CPI
style(ax1,"Core CPI (YoY) vs 2% Target")
ax1.plot(cpi_yoy.index,cpi_yoy, lw=2.2, color=PRIMARY_BLUE, label="Core CPI YoY")
ax1.axhline(2, ls="--", color=MID_GRAY, lw=1.4)
annotate_last(ax1,cpi_yoy,PRIMARY_BLUE)
ax1.yaxis.set_major_formatter(FuncFormatter(lambda x,_: f"{x:.0f}%"))
ax1.legend(frameon=False, loc="upper left")

# Panel 2: Policy expectations proxy
style(ax2,"2Y Treasury Yield (Policy Proxy)")
ax2.plot(ust2y.index, ust2y, lw=2.2, color=PRIMARY_BLUE, label="UST 2Y")
annotate_last(ax2,ust2y,PRIMARY_BLUE,"{:.2f}%")
ax2.yaxis.set_major_formatter(FuncFormatter(lambda x,_: f"{x:.1f}%"))
ax2.legend(frameon=False, loc="upper left")

# Panel 3: Positioning
style(ax3,"Equity Positioning (NAAIM Exposure)")
ax3.plot(naaim.index, naaim, lw=2.2, color=PRIMARY_BLUE, label="NAAIM")
annotate_last(ax3,naaim,PRIMARY_BLUE,"{:.0f}")
ax3.legend(frameon=False, loc="upper left")

# Outer frame
for spine in fig.gca().spines.values():
    spine.set_visible(False)
fig.subplots_adjust(hspace=0.35)
plt.figtext(0.99,0.01,"Source: Lighthouse Macro", ha="right", fontsize=9, color="black")

# Thick outer frame
plt.gcf().patch.set_linewidth(6)
plt.gcf().patch.set_edgecolor(PRIMARY_BLUE)

plt.savefig("lighthouse_macro_dashboard.png", bbox_inches="tight")
plt.show()

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.ticker import FuncFormatter
from fredapi import Fred
from alpha_vantage.timeseries import TimeSeries
import yfinance as yf # Import yfinance
from datetime import datetime

# ---- API KEYS
FRED_API_KEY = os.environ.get("FRED_API_KEY", "7f8e44038ee69c4f78cf71873e85db16")
ALPHA_VANTAGE_API_KEY = os.environ.get("ALPHA_VANTAGE_API_KEY", "IOTZFZG01XK55BHI")

fred = Fred(api_key=FRED_API_KEY)
# ts = TimeSeries(key=ALPHA_VANTAGE_API_KEY, output_format='pandas') # Comment out Alpha Vantage TimeSeries

PRIMARY_BLUE = "#0067db"
SUNSET_ORANGE = "#ff8c42"
DEEP_PURPLE = "#5c3c92"
MID_GRAY = "#6e6e6e"

END_DATE = pd.Timestamp.today()
START_DATE = END_DATE - pd.DateOffset(years=10)

# ------------------ Helpers ------------------
def yoy(series: pd.Series) -> pd.Series:
    return series.pct_change(12) * 100

def last_color_label(ax, s: pd.Series, color, fmt="{:.2f}"):
    if s.empty: return
    ax.scatter(s.index[-1], s.iloc[-1], color=color, zorder=3)
    ax.annotate(fmt.format(s.iloc[-1]),
                xy=(s.index[-1], s.iloc[-1]),
                xytext=(8,0), textcoords="offset points",
                color=color, fontsize=9, weight="bold", va="center")

def add_outer_frame(fig, color=PRIMARY_BLUE, linewidth=8):
    rect = Rectangle((0,0),1,1, transform=fig.transFigure,
                     fill=False, color=color, linewidth=linewidth)
    fig.add_artist(rect)

# ------------------ Data Pull ------------------
# Panel 1: Core CPI YoY
core_cpi = fred.get_series("CPILFESL").dropna()
core_cpi.index = pd.to_datetime(core_cpi.index)
core_cpi = core_cpi.loc[core_cpi.index >= START_DATE]
core_cpi_yoy = yoy(core_cpi).dropna()

# Panel 2: Policy proxy (EFFR - 2Y)
effr = fred.get_series("FEDFUNDS").dropna()
dgs2 = fred.get_series("DGS2").dropna()
effr.index, dgs2.index = pd.to_datetime(effr.index), pd.to_datetime(dgs2.index)
spread = (effr.resample("M").mean() - dgs2.resample("M").mean()) / 0.25

# Panel 3: SPY / TLT
# Fetch S&P 500 data from FRED
spy = fred.get_series("SP500").dropna()
spy.index = pd.to_datetime(spy.index)
spy = spy.loc[spy.index >= START_DATE]

# TLT data will be fetched similarly if needed for the panel

In [ ]:
# --- Styling ---
def style(ax, title):
    ax.set_facecolor("white")
    ax.grid(False)
    for spine in ax.spines.values():
        spine.set_visible(False)
    rect = Rectangle((0,0),1,1, transform=ax.transAxes, fill=False, lw=1.2, edgecolor=LIGHT_BLUE)
    ax.add_patch(rect)
    ax.set_title(title, loc="left", fontsize=12, weight="bold")

def annotate_last(ax, s, color, fmt="{:.1f}%"):
    if s.empty: return
    x,y = s.index[-1], s.iloc[-1]
    ax.scatter([x],[y], color=color, s=25)
    ax.annotate(fmt.format(y),(x,y),xytext=(6,0),textcoords="offset points", color=color, fontsize=9, weight="bold")

# --- Plot ---
fig, (ax1, ax2, ax3) = plt.subplots(3,1,figsize=(10,9), dpi=200)
fig.patch.set_facecolor("white")

# Panel 1: CPI
style(ax1,"Core CPI (YoY) vs 2% Target")
ax1.plot(core_cpi_yoy.index,core_cpi_yoy, lw=2.2, color=PRIMARY_BLUE, label="Core CPI YoY")
ax1.axhline(2, ls="--", color=MID_GRAY, lw=1.4)
annotate_last(ax1,core_cpi_yoy,PRIMARY_BLUE)
ax1.yaxis.set_major_formatter(FuncFormatter(lambda x,_: f"{x:.0f}%"))
ax1.legend(frameon=False, loc="upper left")

# Panel 2: Policy expectations proxy
style(ax2,"Policy Expectations Proxy (EFFR - 2Y Yield Spread)")
ax2.plot(spread.index, spread, lw=2.2, color=PRIMARY_BLUE, label="EFFR - 2Y")
annotate_last(ax2,spread,PRIMARY_BLUE,"{:.2f}")
ax2.yaxis.set_major_formatter(FuncFormatter(lambda x,_: f"{x:.2f}"))
ax2.legend(frameon=False, loc="upper left")

# Panel 3: Equity Positioning (Using SP500 from FRED as proxy)
style(ax3,"Equity Positioning (SP500 Index)")
ax3.plot(spy.index, spy, lw=2.2, color=PRIMARY_BLUE, label="SP500")
annotate_last(ax3,spy,PRIMARY_BLUE,"{:.0f}")
ax3.legend(frameon=False, loc="upper left")


# Outer frame
for spine in fig.gca().spines.values():
    spine.set_visible(False)
fig.subplots_adjust(hspace=0.35)
plt.figtext(0.99,0.01,"Source: Lighthouse Macro", ha="right", fontsize=9, color="black")

# Thick outer frame
plt.gcf().patch.set_linewidth(6)
plt.gcf().patch.set_edgecolor(PRIMARY_BLUE)

plt.savefig("lighthouse_macro_dashboard.png", bbox_inches="tight")
plt.show()